In [23]:
# import sys
# sys.path.append('/tmp/challenges/scyther/crypto-attacks')
# from attacks.factorization import branch_and_prune

c = 8218236802189420259290500179561111010118277541067408364486816176078941700980692366795067861935575821202355571229357296914701529129443717452205232538522060413043502714377818635065207211683899235629132791556690578604926605015405235164602254026658436914312748758201320082818083968704728686453325518000845192282971282282564051903228450084152388488929568795382247930117644047522018215709851795847378667081597574393152299635034606609198054840741349111093991899069141552164038594153083171007363867497171686425109907148831102281256989874896906859620191188384328366898973120428060127472202755334623920382572925381700071252612
n = 16845917380076999552562828814575475978984749768410998999923336296959280214399713823432180364896144400493721660326173603190615734643952957931727929263822289256776033644775943588126170700571160417827628798559652443561985000984461974238525478419074793274427694713739460084852203705389203106966645357894251875540702607372115695389949213672388809257849388934099627704039100699713030911150969114733682541306858363978669951997345659417629023085026307196952106966444414777088012006398051996384604526964035320997863379309389878588982806184509712031465980747256711956894544820705669352530384372805666220916987393468501227726461
leak1 = 89697950374033416355304606625673308472831933610491815518726579310298243707953424357576276605064125986877947558031119876424642243917268996093039001725023065068527235614746832207348952588038619151813238913333750727008843042345694452398791298997642161090710788693938248476212690183035436226762777644851432681182
leak2 = 159747017129553250436615518141206238831341642229591172948077373686937329410774504342340001209917004056839665771963880602659828314141477698162876567643002868293257756988280702756055583954596991013879019668359426293054763061867237961413915820218201090453263517794031142344937845684329298521999750006065593183855
P = 72442321447884130738674369234418803091195416916469006011636832948959268317773983900698278767240824075361677012900380332331418960867255587696032046753680167361246866739162254761122127266401792076243811438411901652193222411316998712036504540721773338658972730306732385214070024518966009472986163307955818464332
Q = 92190301182602923315457459577267706316341786597385450236615268668130181326840788026496259399008511524278315594557092711823984928114821164531812657819805087991417851156238470403573737993356683194791958476210088436664840206949545542117292582568930714508390435517675105139754012024312367945043906618914201182273

# Import necessary functions for SageMath
# from sage.all import *
# If running in a standard Python environment, you might need:
from Crypto.Util.number import isPrime, bytes_to_long, inverse, long_to_bytes
# For standard Python, large integers are handled automatically,
# but you might need a different isPrime implementation or library.
# Assuming SageMath or equivalent environment for isPrime and Integer

# --- Inputs ---
# Replace these with the actual values from the challenge output
# Example placeholder values:
# n = ... # Your RSA modulus n
# leak1 = ... # Your leak1 value
# leak2 = ... # Your leak2 value
# P = ... # Your P value (p & leak1)
# Q = ... # Your Q value (q & leak2)
# c = ... # Your flag ciphertext (needed for decryption after factoring)

# Assume n, leak1, leak2, P, Q, c are defined before running this script

k_bits = 1024 # The bit length of p and q

# --- Preprocessing ---

# Calculate masks of unknown bits (positions where leak is 0)
# Using Sage's Integer to ensure large integer operations are handled correctly
# In Python, `~leak1` might behave differently depending on bit representation.
# This explicit calculation is safer for the mask of 0-bits in leak1.
M1 = (Integer(1) << k_bits) - 1 - Integer(leak1)
M2 = (Integer(1) << k_bits) - 1 - Integer(leak2)

# Identify unknown bit positions
Up = set() # Indices where bit in p is unknown (leak1 bit is 0)
Uq = set() # Indices where bit in q is unknown (leak2 bit is 0)
for i in range(k_bits):
    if (M1 >> i) & 1: # Check if the i-th bit of M1 is 1
        Up.add(i)
    if (M2 >> i) & 1: # Check if the i-th bit of M2 is 1
        Uq.add(i)

# Combined sorted list of unique unknown indices (highest first)
# This determines the order in which we assign unknown bits
U = sorted(list(Up.union(Uq)), reverse=True)

print(f"Total bits: {k_bits}")
print(f"Unknown bits in p: {len(Up)}")
print(f"Unknown bits in q: {len(Uq)}")
print(f"Total unique unknown bits: {len(U)}")
# print(f"Combined unknown indices (highest first): {U[:20]}...") # Print first few for verification

# Calculate initial remaining max sums for p and q
# This is the maximum possible value from the unknown bits
# which is simply the masks M1 and M2 treated as sums of powers of 2
initial_p_rem_max = M1
initial_q_rem_max = M2

# --- Recursive Branch and Prune Function ---

def find_pq(current_p, current_q, u_index, current_p_rem_max, current_q_rem_max):
    """
    Recursive function to find p and q using branch and prune.

    Args:
        current_p: Current partial value of p (P + sum of assigned unknown bits in p).
        current_q: Current partial value of q (Q + sum of assigned unknown bits in q).
        u_index: Current index in the sorted list U.
        current_p_rem_max: Max possible value from the remaining unknown bits in p.
                           (Sum of 2^i for i in U[u_index:] and Up)
        current_q_rem_max: Max possible value from the remaining unknown bits in q.
                           (Sum of 2^j for j in U[u_index:] and Uq)

    Returns:
        A tuple (p, q) if the factors are found, otherwise None.
    """
    global n, Up, Uq # Access global variables

    # Calculate current bounds for p and q
    p_lower_bound = current_p
    p_upper_bound = current_p + current_p_rem_max
    q_lower_bound = current_q
    q_upper_bound = current_q + current_q_rem_max

    # --- Pruning Rules ---
    # If the maximum possible product is less than n, this branch is too low
    if p_upper_bound * q_upper_bound < n:
        # print(f"Pruning low: ({p_upper_bound} * {q_upper_bound}) < {n}")
        return None
    # If the minimum possible product is greater than n, this branch is too high
    if p_lower_bound * q_lower_bound > n:
        # print(f"Pruning high: ({p_lower_bound} * {q_lower_bound}) > {n}")
        return None

    # --- Base Case ---
    # If we have assigned all unknown bits (processed all indices in U)
    if u_index == len(U):
        # Check if the constructed p and q multiply to n
        if current_p * current_q == n:
            # Verify that they are actually prime
            # isPrime is from SageMath or Crypto.Util.number
            if isPrime(current_p) and isPrime(current_q):
                print(f"\n!!! Found factors: p = {current_p}, q = {current_q} !!!")
                return (current_p, current_q)
        # If they don't multiply to n or are not prime, this base case is not a solution
        return None

    # --- Recursive Step ---
    # Get the current index from the sorted list U we are processing
    idx = U[u_index]
    # The remaining indices list for the next recursive call
    next_u_index = u_index + 1

    # Calculate remaining max contributions *after* processing idx
    # Subtract 2^idx if idx was one of the remaining unknown bits in p or q
    next_p_rem_max = current_p_rem_max - (Integer(1) << idx if idx in Up else 0)
    next_q_rem_max = current_q_rem_max - (Integer(1) << idx if idx in Uq else 0)

    # Branching based on whether idx is an unknown bit position for p, q, or both
    if idx in Up and idx in Uq:
        # Case 3: idx is an unknown bit position for BOTH p and q
        # We need to try all 4 combinations for the bits p_idx and q_idx (0/1)
        # print(f"Branching idx={idx} (p, q unknown)")

        # Try (p_idx=0, q_idx=0)
        result = find_pq(current_p, current_q, next_u_index, next_p_rem_max, next_q_rem_max)
        if result: return result # If solution found in this branch, return it

        # Try (p_idx=0, q_idx=1)
        result = find_pq(current_p, current_q + (Integer(1) << idx), next_u_index, next_p_rem_max, next_q_rem_max)
        if result: return result

        # Try (p_idx=1, q_idx=0)
        result = find_pq(current_p + (Integer(1) << idx), current_q, next_u_index, next_p_rem_max, next_q_rem_max)
        if result: return result

        # Try (p_idx=1, q_idx=1)
        result = find_pq(current_p + (Integer(1) << idx), current_q + (Integer(1) << idx), next_u_index, next_p_rem_max, next_q_rem_max)
        if result: return result

    elif idx in Up:
        # Case 1: idx is an unknown bit position ONLY for p
        # We need to try both combinations for the bit p_idx (0/1)
        # print(f"Branching idx={idx} (p unknown)")

        # Try p_idx = 0
        result = find_pq(current_p, current_q, next_u_index, next_p_rem_max, current_q_rem_max) # q_rem_max doesn't change as idx not in Uq
        if result: return result

        # Try p_idx = 1
        result = find_pq(current_p + (Integer(1) << idx), current_q, next_u_index, next_p_rem_max, current_q_rem_max)
        if result: return result

    elif idx in Uq:
        # Case 2: idx is an unknown bit position ONLY for q
        # We need to try both combinations for the bit q_idx (0/1)
        # print(f"Branching idx={idx} (q unknown)")

        # Try q_idx = 0
        result = find_pq(current_p, current_q, next_u_index, current_p_rem_max, next_q_rem_max) # p_rem_max doesn't change as idx not in Up
        if result: return result

        # Try q_idx = 1
        result = find_pq(current_p, current_q + (Integer(1) << idx), next_u_index, current_p_rem_max, next_q_rem_max)
        if result: return result
    # Note: If idx is in neither Up nor Uq, it should not be in the combined list U.

    # If none of the branches from this index led to a solution
    return None

# --- Main Execution Flow ---

# Assuming n, leak1, leak2, P, Q, c are loaded/defined here.
# For example, you'd parse these from the challenge output.
# Example usage (replace with actual challenge data):
# from your_challenge_loader import get_challenge_data # Placeholder

# n, c, leak1, leak2, P, Q = get_challenge_data() # Placeholder

# Ensure inputs are Sage Integers if not already
n = Integer(n)
leak1 = Integer(leak1)
leak2 = Integer(leak2)
P = Integer(P)
Q = Integer(Q)
# c = Integer(c) # Need c for decryption later

print("Starting branch and prune algorithm...")

# Initial call to the recursive function
# We start with the known parts P and Q, the full list of unknown indices U,
# and the initial maximum remaining values M1 and M2.
solution = find_pq(P, Q, 0, initial_p_rem_max, initial_q_rem_max)

if solution:
    p_found, q_found = solution
    print("\nBranch and Prune finished.")
    print("Factors found:")
    print("p =", p_found)
    print("q =", q_found)

    # Verify the product
    print("Verification: p*q =", p_found * q_found)
    print("Original n =", n)
    if p_found * q_found == n:
        print("Product matches n. Factoring successful!")

        # --- Decrypt the Flag ---
        # Assuming e = 65537 is standard for this challenge
        e = 65537
        print(f"\nUsing e = {e} to decrypt flag with ciphertext c = {c}...")

        # Calculate phi(n) = (p-1)(q-1)
        phi = (p_found - 1) * (q_found - 1)
        print("Calculated phi(n) =", phi)

        # Calculate the private exponent d = e^-1 mod phi(n)
        # Use Sage's inverse_mod or Crypto.Util.number.inverse
        d = inverse(e, phi)
        print("Calculated d =", d)

        # Decrypt the ciphertext c
        flag_int = pow(c, d, n)
        print("Decrypted flag (integer) =", flag_int)

        # Convert the integer to bytes
        flag_bytes = long_to_bytes(flag_int)
        print("\n" + "="*40)
        print("Decrypted Flag:", flag_bytes.decode('utf-8', errors='ignore')) # Decode, ignoring potential non-UTF8 chars
        print("="*40)

    else:
        print("Error: Found factors do not multiply to n.")

else:
    print("\nBranch and prune algorithm finished but could not find factors.")
    print("Consider increasing the search depth or adding more pruning rules if applicable.")

Total bits: 1024
Unknown bits in p: 237
Unknown bits in q: 285
Total unique unknown bits: 454
Starting branch and prune algorithm...

!!! Found factors: p = 162326978202771881407614334053706233651394820172069197429981228196524492056937521295733945540439586562450603275070473174497200285787505873068320193310639791303057609577492070980760497601191880055310238147943082108348411210224918437273918028587641582726570928801765062447944418879127387872602154337263148242029, q = 103777681113695124078632347038617809988850376283155052257777583017656792294781831102710726829882261793441000380659344626306361975735442118307207318530125771390777403621551836766183615670023045643975266416201685130733465351613454231704595865542560853414293074519391369944898758974319882653810193578909194825809 !!!

Branch and Prune finished.
Factors found:
p = 16232697820277188140761433405370623365139482017206919742998122819652449205693752129573394554043958656245060327507047317449720028578750587306832019331063979130305760957